In [4]:
import traceback #Для отладки
from IPython.display import clear_output #Для интерфейса
import pickle #Для сохранения/загрузки

class ChessField(): #Класс поля
    def __init__(self, PlayerWhite, PlayerBlack):
        self.gameMap = [[' ', '|', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', '|', ' '],
                        ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
                       ['8', '|', ' ', '█', ' ', '█', ' ', '█', ' ', '█', '|', '8'],
                       ['7', '|', '█', ' ', '█', ' ', '█', ' ', '█', ' ', '|', '7'],
                       ['6', '|', ' ', '█', ' ', '█', ' ', '█', ' ', '█', '|', '6'],
                       ['5', '|', '█', ' ', '█', ' ', '█', ' ', '█', ' ', '|', '5'],
                       ['4', '|', ' ', '█', ' ', '█', ' ', '█', ' ', '█', '|', '4'],
                       ['3', '|', '█', ' ', '█', ' ', '█', ' ', '█', ' ', '|', '3'],
                       ['2', '|', ' ', '█', ' ', '█', ' ', '█', ' ', '█', '|', '2'],
                       ['1', '|', '█', ' ', '█', ' ', '█', ' ', '█', ' ', '|', '1'],
                        ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
                       [' ', '|', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', '|', ' ']]
        self.gameMap[3] = ['7', '|'] + PlayerBlack.figures[0] + ['|', '7']
        self.gameMap[2] = ['8', '|'] + PlayerBlack.figures[1] + ['|', '8']
        self.gameMap[8] = ['2', '|'] + PlayerWhite.figures[0] + ['|', '2']
        self.gameMap[9] = ['1', '|'] + PlayerWhite.figures[1] + ['|', '1']
        self.__gameTrack = 1
        
    def UpdateMap(self, playerSide, CurrentChoice, choiceStart): #Обновление поля
        dangerZone = []
        if CurrentChoice == True:
            if self.gameMap[choiceStart[1]][choiceStart[0]].figureSide != playerSide:
                print("Нельзя брать чужую фигуру!")
                return False
            else:
                dangerZone = self.gameMap[choiceStart[1]][choiceStart[0]].listOfMoves
        printMap = ''
        temp_i = 0
        temp_j = 0
        for i in self.gameMap:
            for j in i:
                if type(j) in Figure.__subclasses__():
                    j.listOfMoves = self.__MovementLockCheck([temp_j, temp_i])
                    j.currentPosition = [temp_j, temp_i]
                    if CurrentChoice == False and j.figureSide != playerSide:
                        dangerZone.append(j.listOfMoves)
                    elif CurrentChoice == True and [temp_j, temp_i] in dangerZone:
                        printMap += "\033[31m{}".format(j.figureMask) + '\033[0m '
                    else:
                        printMap += j.figureMask + ' '
                else:
                    if CurrentChoice == True and [temp_j, temp_i] in dangerZone:
                        printMap += "\033[33m\033[43m{}".format(j) + '\033[0m '
                    else:
                        printMap += j + ' '
                temp_j += 1
            printMap += '\n'
            temp_j = 0
            temp_i += 1
        if CurrentChoice == False:
            printMap = ''
            for i in self.gameMap:
                for j in i:
                    if type(j) in Figure.__subclasses__():
                        if j.figureSide == playerSide:
                            inside = False
                            for q in dangerZone:
                                if j.currentPosition in q:
                                    inside = True
                                    break
                            if inside == True:
                                printMap += "\033[31m{}".format(j.figureMask) + '\033[0m '
                            else:
                                printMap += j.figureMask + ' ' 
                        else:
                            printMap += j.figureMask + ' '  
                    else:
                        printMap += j + ' '
                printMap += '\n'        
        print(printMap)
        print("Ход: ", self.__gameTrack)
        return True
        
    
    def MovingFigure(self, choiceStart, choiceEnd, side, otherPlayer): #Движение фигуры
        try:
            if choiceEnd in self.gameMap[choiceStart[1]][choiceStart[0]].listOfMoves:
                if isinstance(self.gameMap[choiceStart[1]][choiceStart[0]], Pawn) and len(self.gameMap[choiceStart[1]][choiceStart[0]].moveRules) == 4:
                    self.gameMap[choiceStart[1]][choiceStart[0]].moveRules.pop(3)
                self.gameMap[choiceEnd[1]][choiceEnd[0]] = self.gameMap[choiceStart[1]][choiceStart[0]]
                self.__gameTrack += 1
                if (choiceStart[1] % 2 != 0  and choiceStart[0] % 2 == 0) or (choiceStart[1] % 2 == 0 and choiceStart[0] % 2 != 0):
                    self.gameMap[choiceStart[1]][choiceStart[0]] = "█"
                else:
                    self.gameMap[choiceStart[1]][choiceStart[0]] = " "
                if isinstance(self.gameMap[choiceEnd[1]][choiceEnd[0]], Pawn):
                    self.gameMap[choiceEnd[1]][choiceEnd[0]] = self.gameMap[choiceEnd[1]][choiceEnd[0]].UniqueMove(choiceEnd)
                return True
            return False        
        except:
            print("Неправильно введеное значение!")
            return False
            
    def __MovementLockCheck(self, choiceStart): #Проверка возможностей хода
            possibleMoves = []
            currentStartPoint = self.gameMap[choiceStart[1]][choiceStart[0]]
            for i in currentStartPoint.moveRules:                 
                j = 0
                tempChoice = choiceStart
                tempChoice = tempChoice.copy()
                while (currentStartPoint.distance != j): 
                    tempChoice[1] += i[0]
                    tempChoice[0] += i[1] 
                    if (tempChoice[1] < 2 or tempChoice[1] > 9) or (tempChoice[0] < 2 or tempChoice[0] > 9):
                        break
                    elif type(self.gameMap[tempChoice[1]][tempChoice[0]]) in Figure.__subclasses__():
                        if self.gameMap[tempChoice[1]][tempChoice[0]].figureSide != currentStartPoint.figureSide:
                            if isinstance(currentStartPoint, Pawn) and (i == [1, 0] or i == [2, 0] or i == [-1, 0] or i == [-2, 0]):
                                break
                            possibleMoves.append(tempChoice.copy())
                        break
                    elif isinstance(currentStartPoint, Pawn) and (i == [1, 1] or i == [-1, -1] or i == [-1, 1] or i == [1, -1]):
                        break
                    possibleMoves.append(tempChoice.copy())
                    j += 1
            return possibleMoves
        
        
    

class Player(): #Класс игрока
    def __init__(self, figures, side):
        self.figures = figures
        self.side = side
    
    def CurrentTurn(self): #Текущий ход
        try:
            print("Сторона: ", self.side)
            choiceStartIn = input("Выберите фигуру (напишите 0, чтобы сдаться; 1, чтобы сохранить и выйти): ")
            if choiceStartIn == '0':
                 return 0
            elif choiceStartIn == '1':
                return 2
            choiceStart = [Player.LetterChanger(choiceStartIn[0]), 12 - (int(choiceStartIn[1]) + 2)]
            clear_output()
            if newGame.UpdateMap(self.side, True, choiceStart) == False:   
                return 1
            choiceEndIn = input("Выберите куда поставить (напишите 0, чтобы отменить ход): ")
            if choiceEndIn == '0':
                clear_output()
                return 1
            choiceEnd = [Player.LetterChanger(choiceEndIn[0]), 12 - (int(choiceEndIn[1]) + 2)]
            clear_output()
            if len(choiceStartIn) > 2 or len(choiceEndIn) > 2:
                raise ValueError
        except:
            return [-1, -1]
        return [choiceStart, choiceEnd]
        
    def LetterChanger(letter): #Замена буквы-цифры
        match letter:
            case "A": return 2
            case "B": return 3
            case "C": return 4
            case "D": return 5
            case "E": return 6
            case "F": return 7
            case "G": return 8
            case "H": return 9
            case _: return -1
                
    
class Figure(): #Основа фигуры
    def __init__(self, figureSide):
        self.figureSide = figureSide
        self.currentPosition = []
        self.listOfMoves = []
            
        
class Pawn(Figure): #Пешка
    def __init__(self, side, figureMask):
        if side == 'white':
            self.moveRules = [[-1, -1], [-1, 0], [-1, 1], [-2, 0]]
        else:
            self.moveRules = [[1, -1], [1, 0], [1, 1], [2, 0]]
        self.distance = 1
        self.moveStart = True
        self.figureMask = figureMask
        super().__init__(side)
        
    def UniqueMove(self, currPos):
        if currPos[1] == 2 or currPos[1] == 9:
            while(True):
                answ = input("Введите желаемую фигуру: ")
                if self.figureSide == 'white':
                    match answ:
                        case "Королева": return Queen(self.figureSide, "Q")
                        case "Конь": return Knight(self.figureSide, "N")
                        case "Слон": return Bishop(self.figureSide, "B")
                        case "Ладья": return Rook(self.figureSide, "R")
                        case _: continue
                else:
                    match input("Введите желаемую фигуру: "):
                        case "Королева": return Queen(self.figureSide, "q")
                        case "Конь": return Knight(self.figureSide, "n")
                        case "Слон": return Bishop(self.figureSide, "b")
                        case "Ладья": return Rook(self.figureSide, "r")
                        case _: continue
        else:
            return self
        
class Knight(Figure): #Конь
    def __init__(self, side, figureMask):
        self.moveRules = [[-2, 1], [-2, -1], [-1, 2], [1, 2], [2, 1], [2, -1], [1, -2], [-1, -2]]
        self.distance = 1
        self.figureMask = figureMask
        super().__init__(side)    
            
class Rook(Figure): #Ладья
    def __init__(self, side, figureMask):
        self.moveRules = [[-1, 0], [0, 1],  [1, 0],  [0, -1]]
        self.distance = 8
        self.figureMask = figureMask
        super().__init__(side)
        
        
class King(Figure): #Король
    def __init__(self, side, figureMask):
        self.moveRules = [[-1, 0],[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1]]
        self.distance = 8
        self.figureMask = figureMask
        self.check = False
        self.checkmate = False
        self.unimove = True
        super().__init__(side)

class Queen(Figure): #Королева
    def __init__(self, side, figureMask):
        self.moveRules = [[-1, 0],[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1]]
        self.distance = 8
        self.figureMask = figureMask
        super().__init__(side)
        
        

class Bishop(Figure): #Слон
    def __init__(self, side, figureMask):
        self.moveRules = [[-1, 1], [1, 1], [1, -1], [-1, -1]]
        self.distance = 8
        self.figureMask = figureMask
        super().__init__(side)

        
def SaveGame(Player1, Player2, newGame, otkat):
    savearr = [Player1, Player2, newGame]
    otkat.append(savearr)
    with open('save.dat', "wb") as file:
        pickle.dump(otkat, file)
        file.close()
    return otkat

def LoadGame():
    loadarr = []
    try:
        with open('save.dat', "rb") as file:
            loadarr = pickle.load(file)
            file.close()
        return loadarr
    except:
        print('Сохранений нет!')
        
    
    

newGameChoice = input("Новая игра или сохранение?")
if newGameChoice == "Новая игра":
    Player1 = Player([[Pawn("white", "P") for i in range(8)],[Rook("white", "R"), Knight("white", "N"), Bishop("white", "B"), Queen("white", "Q"), King("white", "K"), Bishop("white", "B"), Knight("white", "N"), Rook("white", "R"),]], "white")
    Player2 = Player([[Pawn("black", "p") for i in range(8)],[Rook("black", "r"), Knight("black", "n"), Bishop("black", "b"), Queen("black", "q"), King("black", "k"), Bishop("black", "b"), Knight("black", "n"), Rook("black", "r"),]], "black")
    newGame = ChessField(Player1, Player2)
    otkatarr = [[Player1, Player2, newGame]]
else:
    otkatarr = LoadGame()
    Player1 = otkatarr[len(otkatarr) - 1][0]
    Player2 = otkatarr[len(otkatarr) - 1][1]
    newGame = otkatarr[len(otkatarr) - 1][2]

    
while (True):
    try:
        while (True):
            newGame.UpdateMap(Player1.side, False, [-1, -1])
            if Player1.figures[1][4].checkmate == True:
                print(Player1.side, ' проиграли!')
                raise SystemExit
            choice = Player1.CurrentTurn()
            if choice == 0:
                print(Player1.side, ' сдались!')
                raise SystemExit
            elif choice == 1:
                continue
            elif choice == 2:
                SaveGame(Player1, Player2, newGame)
                print("Игра сохранена")
                raise SystemExit
            if newGame.MovingFigure(choice[0], choice[1], Player1.side, Player2) == True:
                break
        otkatarr = SaveGame(Player1, Player2, newGame, otkatarr)
        while (True):
            newGame.UpdateMap(Player2.side, False, [-1, -1])
            if Player2.figures[1][4].checkmate == True:
                print(Player2.side, ' проиграли!')
                raise SystemExit
            choice = Player2.CurrentTurn()
            if choice == 0:
                print(Player2.side, ' сдались!')
                raise SystemExit
            elif choice == 1:
                continue
            elif choice == 2:
                SaveGame(Player1, Player2, newGame, otkatarr)
                print("Игра сохранена")
                raise SystemExit
            if newGame.MovingFigure(choice[0], choice[1], Player2.side, Player1) == True:
                break
        otkatarr = SaveGame(Player1, Player2, newGame, otkatarr)
    except:
        print("Конец игры!")
        break


Новая игра или сохранение?ыв
Сохранений нет!


TypeError: object of type 'NoneType' has no len()